In [1]:
%load_ext autoreload
#To update the imported files

In [2]:
%autoreload
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error
from utilities import *
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [19]:
#Loading the y-paramter from file, might be changed to y_a etc.
train_a = pd.read_parquet('../Data_and_task/A/train_targets.parquet')
train_b = pd.read_parquet('../Data_and_task/B/train_targets.parquet')
train_c = pd.read_parquet('../Data_and_task/C/train_targets.parquet')

In [20]:
print(train_b.shape[0])
train_b = train_b.dropna()
print(train_b.shape[0])
print(train_c.shape[0])
train_c = train_c.dropna()
print(train_c.shape[0])

32848
32844
32155
26095


In [4]:
#Loading estimated/forecasted training_weather from file
X_train_estimated_a = pd.read_parquet('../Data_and_task/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../Data_and_task/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../Data_and_task/C/X_train_estimated.parquet')

In [5]:
#Loading observed weather from file
X_train_observed_a = pd.read_parquet('../Data_and_task/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../Data_and_task/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../Data_and_task/C/X_train_observed.parquet')

In [31]:
X_train_observed_c.describe()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
count,116825,116825.000000,116825.000000,96902.000000,1.168250e+05,116825.000000,108313.000000,116825.000000,116825.000000,116825.000000,1.168250e+05,116825.000000,1.168250e+05,116825.000000,116825.0,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,9232.0,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000,116825.000000
mean,2020-08-31 11:00:00,6.013866,1.253214,2605.747559,5.423583e+05,150.655075,1710.178467,-0.013867,275.147400,40.920612,1.473144e+05,52.393135,1.886154e+05,66.632774,24.0,0.186821,0.015556,0.373644,0.046708,0.093405,0.495219,0.583403,1009.408325,0.005616,0.084049,993.936462,1000.055786,2.129328,0.009322,73.668045,1006.202332,250.0,0.560316,0.000026,0.000657,0.089608,179.576492,-0.334226,0.062269,279.682556,73.351135,33186.710938,2.855492,0.739988,0.838296,0.000211
min,2019-01-01 00:00:00,0.500000,1.139000,27.799999,0.000000e+00,0.000000,27.900000,-1.000000,247.300003,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,24.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,944.299988,0.000000,0.000000,929.799988,935.599976,0.000000,0.000000,18.700001,941.400024,250.0,0.000000,0.000000,-0.000000,0.000000,0.003000,-50.042000,0.000000,257.899994,0.000000,130.600006,0.000000,-6.300000,-8.800000,-0.100000
25%,2019-11-01 05:30:00,4.000000,1.228000,981.400024,0.000000e+00,0.000000,618.500000,0.000000,270.600006,0.000000,0.000000e+00,0.000000,0.000000e+00,40.799999,24.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1001.500000,0.000000,0.000000,986.200012,992.200012,0.000000,0.000000,63.900002,998.400024,250.0,0.000000,0.000000,0.000000,0.000000,89.529999,-17.424999,0.000000,275.000000,52.599998,16388.000000,1.600000,-0.900000,-0.500000,0.000000
50%,2020-08-31 11:00:00,5.400000,1.253000,1643.000000,1.267980e+04,0.000000,1178.400024,0.000000,274.899994,0.000000,1.394200e+04,0.000000,0.000000e+00,80.199997,24.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1010.299988,0.000000,0.000000,994.799988,1000.900024,0.000000,0.000000,76.000000,1007.000000,250.0,0.000000,0.000000,0.000000,0.000000,179.462997,-0.288000,0.000000,278.899994,93.300003,37765.199219,2.500000,0.400000,0.900000,0.000000
75%,2021-07-01 16:30:00,7.900000,1.277000,3406.175049,8.698632e+05,242.000000,2056.399902,0.000000,280.600006,68.900002,2.478511e+05,33.400002,1.257561e+05,98.699997,24.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1018.299988,0.000000,0.000000,1002.799988,1008.900024,0.000000,0.000000,85.699997,1015.099976,250.0,0.000000,0.000000,0.000000,0.100000,269.747009,16.603001,0.100000,284.299988,99.900002,48576.000000,3.800000,2.300000,2.000000,0.000000
max,2022-05-01 22:00:00,18.900000,1.441000,12431.299805,3.007722e+06,837.000000,11665.700195,1.000000,295.100006,344.500000,1.217465e+06,689.799988,2.446962e+06,100.000000,24.0,37.599998,7.700000,37.599998,20.799999,34.099998,1.000000,1.

In [25]:
X_train_observed_a["snow_drift:idx"].dropna()

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
118664    0.0
118665    0.0
118666    0.0
118667    0.0
118668    0.0
Name: snow_drift:idx, Length: 118669, dtype: float32

In [6]:
#Loading estimated/forecasted test_weather from file
X_test_estimated_a = pd.read_parquet('../Data_and_task/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../Data_and_task/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../Data_and_task/C/X_test_estimated.parquet')

In [10]:
#Cleaning data such that only wanted features are included
selected_features = [
    "date_forecast", "clear_sky_rad:W", "diffuse_rad:W", "direct_rad:W",
    "absolute_humidity_2m:gm3", "effective_cloud_cover:p",
    "is_day:idx", "is_in_shadow:idx", "fresh_snow_12h:cm", "fresh_snow_1h:cm",
    "fresh_snow_24h:cm", "fresh_snow_3h:cm", "fresh_snow_6h:cm",
    "snow_depth:cm", "snow_melt_10min:mm", "sun_azimuth:d", 
    "sun_elevation:d", "wind_speed_10m:ms"
]
#Defining y_fetures, will always be the same
y_features = ["time", "pv_measurement"]

X_train_observed_a_clean = X_train_observed_a[selected_features].copy()
X_train_estimated_a_clean = X_train_estimated_a[selected_features].copy()

X_test_estimated_a_clean = X_test_estimated_a[selected_features][::4].drop(columns = ["date_forecast"]).copy()
X_test_estimated_b_clean = X_test_estimated_b[selected_features][::4].drop(columns = ["date_forecast"]).copy()
X_test_estimated_c_clean = X_test_estimated_c[selected_features][::4].drop(columns = ["date_forecast"]).copy()

X_train_observed_b_clean = X_train_observed_b[selected_features].copy()
X_train_estimated_b_clean = X_train_estimated_b[selected_features].copy()
X_train_observed_c_clean = X_train_observed_c[selected_features].copy()
X_train_estimated_c_clean = X_train_estimated_c[selected_features].copy()

In [8]:
#Training a Linear regression model on X_observed_a and testing it on X_estimated_a and evaluating it on MAE, PURELY for testing!
#See below for how its done when submitting
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_clean, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_clean, train_a, "date_forecast", y_features)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_a_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_a_resized)

#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, y_train_observed_a["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(y_train_estimated_a["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 382.9977125715309


/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623

In [9]:
#Training model, running infernece on testing data and saving it in a csv file ready for submission
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_clean, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_clean, train_a, "date_forecast", y_features)

#Merging observed and estimated data to one big df
X_train_a = pd.concat([X_train_observed_a_resized, X_train_estimated_a_resized], ignore_index=True)
y_train_a = pd.concat([y_train_observed_a, y_train_estimated_a], ignore_index=True)
X_test = pd.concat([X_test_estimated_a_clean, X_test_estimated_b_clean, X_test_estimated_c_clean], ignore_index=True)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_a_scaled = scaler.fit_transform(X_train_a)
X_test_a_scaled=scaler.fit_transform(X_test)

#Training the models
#Model A:
reg_a = LinearRegression()
reg_a.fit(X_train_a, y_train_a["pv_measurement"])

# Make predictions for A,B,C on the model trained on only A. Problems with the B and C datasets which need to be explored
y_pred = reg_a.predict(X_test_a_scaled)

#Saving y_pred in a proper csv file
filename = "CSV/second_submission.csv"
submission(filename, y_pred)

/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623

In [18]:
#Training model, running infernece on testing data and saving it in a csv file ready for submission
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_clean, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_clean, train_a, "date_forecast", y_features)

X_train_observed_b_resized, y_train_observed_b = resize_trainingdata(X_train_observed_b_clean, train_b, "date_forecast", y_features)
X_train_estimated_b_resized, y_train_estimated_b = resize_trainingdata(X_train_estimated_b_clean, train_b, "date_forecast", y_features)

X_train_observed_c_resized, y_train_observed_c = resize_trainingdata(X_train_observed_c_clean, train_c, "date_forecast", y_features)
X_train_estimated_c_resized, y_train_estimated_c = resize_trainingdata(X_train_estimated_c_clean, train_c, "date_forecast", y_features)

#Merging observed and estimated data to one big df
X_train_a = pd.concat([X_train_observed_a_resized, X_train_estimated_a_resized], ignore_index=True)
y_train_a = pd.concat([y_train_observed_a, y_train_estimated_a], ignore_index=True)

X_train_b = pd.concat([X_train_observed_b_resized, X_train_estimated_b_resized], ignore_index=True)
y_train_b = pd.concat([y_train_observed_b, y_train_estimated_b], ignore_index=True)

X_train_c = pd.concat([X_train_observed_c_resized, X_train_estimated_c_resized], ignore_index=True)
y_train_c = pd.concat([y_train_observed_c, y_train_estimated_c], ignore_index=True)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_a_scaled = scaler.fit_transform(X_train_a)
X_test_a_scaled=scaler.fit_transform(X_test_estimated_a_clean)

X_train_b_scaled = scaler.fit_transform(X_train_b)
X_test_b_scaled=scaler.fit_transform(X_test_estimated_b_clean)

X_train_c_scaled = scaler.fit_transform(X_train_b)
X_test_c_scaled=scaler.fit_transform(X_test_estimated_b_clean)


#Training the models
#Model A:
reg_a = LinearRegression()
reg_a.fit(X_train_a, y_train_a["pv_measurement"])

# Make predictions
y_pred_a = reg_a.predict(X_test_a_scaled)

#Model B:
reg_b = LinearRegression()
reg_b.fit(X_train_b, y_train_b["pv_measurement"])

# Make predictions
y_pred_b = reg_b.predict(X_test_b_scaled)

#Model C:
reg_c = LinearRegression()
reg_c.fit(X_train_a, y_train_a["pv_measurement"])

# Make predictions
y_pred_c = reg_c.predict(X_test_c_scaled)

y_pred = np.concatenate([y_pred_a, y_pred_b, y_pred_c])
filename = "CSV/second_submission.csv"
submission(filename, y_pred)

/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623